In [1]:
import pandas as pd
import numpy as np

# header=None，第一行不作为head，作为数据
dataset = pd.read_csv('./Market_Basket_Optimisation.csv', header = None) 
print(dataset.shape) # shape为(7501,20)
dataset


(7501, 20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,butter,light mayo,fresh bread,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7497,burgers,frozen vegetables,eggs,french fries,magazines,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7498,chicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7499,escalope,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# 第一种方法：使用efficient_apriori
from efficient_apriori import apriori

# 将数据存放到transactions
transactions = []
# 按照行进行遍历
for i in range(0,dataset.shape[0]):
    # 记录一行transaction
    temp = []
    # 按照列进行遍历
    for j in range(0, dataset.shape[1]):
        if str(dataset.values[i, j]) != 'nan':
           temp.append(str(dataset.values[i, j]))
    transactions.append(temp)   

# print(transactions)

# 挖掘频繁项集和关联规则
itemsets, rules = apriori(transactions, min_support=0.02, min_confidence=0.4)
print("频繁项集：", itemsets)
print("关联规则：", rules)

频繁项集： {1: {('shrimp',): 536, ('vegetables mix',): 193, ('energy drink',): 200, ('almonds',): 153, ('mineral water',): 1788, ('salmon',): 319, ('green tea',): 991, ('low fat yogurt',): 574, ('avocado',): 250, ('cottage cheese',): 239, ('honey',): 356, ('olive oil',): 494, ('frozen smoothie',): 475, ('tomato juice',): 228, ('burgers',): 654, ('eggs',): 1348, ('meatballs',): 157, ('turkey',): 469, ('milk',): 972, ('whole wheat rice',): 439, ('energy bar',): 203, ('french fries',): 1282, ('whole wheat pasta',): 221, ('soup',): 379, ('frozen vegetables',): 715, ('spaghetti',): 1306, ('cookies',): 603, ('cooking oil',): 383, ('champagne',): 351, ('chicken',): 450, ('oil',): 173, ('chocolate',): 1229, ('tomatoes',): 513, ('fresh tuna',): 167, ('red wine',): 211, ('pepper',): 199, ('ham',): 199, ('pancakes',): 713, ('grated cheese',): 393, ('fresh bread',): 323, ('escalope',): 595, ('ground beef',): 737, ('herb & pepper',): 371, ('strawberries',): 160, ('cake',): 608, ('hot dogs',): 243, ('bro

In [3]:
# 第二种方法：使用mlxtend.frequent_patterns

# 处理交易数据，用于get_dummies
# 问题 1：如果先将dataset使用get_dummies，然后会产生重复的列名，导致apriori计算错误，那么get_dummies之后的数据应如何处理？
# 问题 1：transaction_onehot = pd.get_dummies(dataset, prefix = '', prefix_sep = '') -> 7501 rows × 1269 columns
# 问题 1：也就是说如何合并同名的列，重复列名只剩一个，列元素求和？

# 问题2：下面数据处理的代码，有没有可以优化的地方？请老师帮忙修改一下，谢谢！

# 数据处理**********************
trans = []
# 按照行遍历
for i in range(0,dataset.shape[0]):
    temp_string = ''
    # 按照列遍历
    for j in range(0,dataset.shape[1]):
        if str(dataset.at[i,j]) != 'nan':
            temp_string = temp_string +'-'+ dataset.at[i,j]
    trans.append(temp_string)

# 将交易数据存储到transaction
transaction = pd.DataFrame({'Items':trans})
# 将交易数据做one-hot编码
transaction_onehot = transaction['Items'].str.get_dummies('-')
# 数据处理**********************

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# 挖掘频繁项集和关联规则
transaction_ml = transaction_onehot
itemsets_ml = apriori(transaction_ml, min_support = 0.02, use_colnames = True)
print("频繁项集：", itemsets_ml)

rules_ml = association_rules(itemsets_ml, metric = 'lift', min_threshold = 1)
print("关联规则：", rules_ml[ (rules_ml['lift'] >= 1) & (rules_ml['confidence'] >= 0.4) ])


频繁项集：       support                           itemsets
0    0.020397                          (almonds)
1    0.033329                          (avocado)
2    0.033729                         (brownies)
3    0.087188                          (burgers)
4    0.030129                           (butter)
..        ...                                ...
98   0.020131  (mineral water, whole wheat rice)
99   0.022930             (spaghetti, olive oil)
100  0.025197              (spaghetti, pancakes)
101  0.021197                (spaghetti, shrimp)
102  0.020931              (spaghetti, tomatoes)

[103 rows x 2 columns]
关联规则：       antecedents      consequents  antecedent support  consequent support  \
63  (ground beef)  (mineral water)            0.098254            0.238368   
73    (olive oil)  (mineral water)            0.065858            0.238368   
79         (soup)  (mineral water)            0.050527            0.238368   

     support  confidence      lift  leverage  conviction  
63  